# lesson 3

本节主要是学习如何使用Shapely, Geopandas和其他相关库管理和分析空间数据:

- Geocoding / Geocoding in Geopandas
- Conduct a Point in Polygon queries
- How to boost spatial queries using spatial index
- Make spatial and table joins between layers
- Nearest neighbour analysis

## 学习目标

- 执行geocoding，转换地址为坐标
- 执行point-in-polygon查询
- 从WFS服务和KML文件中读取数据
- 空间连接
- 执行最近邻分析，即找到最近的点

## Geocoding

geocoding，比如地址和Points之间的转换，是非常常见的GIS任务。又非常幸运的是，在python中有一些很nice的库使geocoding很容易。比如[geopy](http://geopy.readthedocs.io/en/1.11.0/)，该库通过使用第三方的geocoders和其他数据源可以很轻松的定位全球地址，城市，国家和地标的坐标。这些第三方geocoders包括：

- [ESRI ArcGIS](https://developers.arcgis.com/rest/geocode/api-reference/overview-world-geocoding-service.htm)
- [Baidu Maps](http://lbsyun.baidu.com/index.php?title=webapi/guide/webservice-geocoding)
- [Bing](https://msdn.microsoft.com/en-us/library/ff701715.aspx)
- [GeocodeFarm](https://www.geocode.farm/geocoding/free-api-documentation/)
- [GeoNames](http://www.geonames.org/export/geonames-search.html)
- [Google Geocoding API (V3)](https://developers.google.com/maps/documentation/geocoding/)
- [HERE](https://developer.here.com/documentation/geocoder/)
- [IGN France](https://geoservices.ign.fr/documentation/geoservices/index.html)
- [Mapquest](https://developer.mapquest.com/documentation/open/)
- [OpenCage](https://opencagedata.com/api)
- [OpenMapQuest](http://developer.mapquest.com/web/products/open/geocoding-service)
- [Open Street Map Nominatim](https://wiki.openstreetmap.org/wiki/Nominatim)
- [What3words](https://developer.what3words.com/public-api/docsv2#overview)
- [Yandex](https://tech.yandex.com/maps/doc/geocoder/desc/concepts/input_params-docpage/)

这些源暂时不一一记录了。总之有很多geocoders可选，他们之间有所不同。不过为了使用服务，需要从服务商那里申请API访问keys，比如google的Geocoding API，需要在谷歌云的[API控制台](https://code.google.com/apis/console)上创建项目并启动一个来自[库](https://console.developers.google.com/apis/library)的API，对google的话，在库中搜索geocoder，会出来Geocoding API，启用它即可。这里有一个如何使用Google API Console的[说明](https://developers.googleblog.com/2016/03/introducing-google-api-console.html)。使用geocoder之前需要获取google的geocoding key api，这部分可以参考谷歌文档[Get an API Key](https://developers.google.com/maps/documentation/geocoding/get-api-key)。按照文档步骤一步步即可。

不过这类服务有rate limiting，短时间内的请求太多会报错。不过可以查询[geopy文档](https://geopy.readthedocs.io/en/stable/#usage-with-pandas)看看如何处理rate limiting。

更多geocoders信息可参考:[Geocoder: Simple, Consistent](https://geocoder.readthedocs.io/)

这里选择使用Nominatim geocoder来定位较小数目的地址。该服务允许每秒1次的请求，可参考[文档](https://operations.osmfoundation.org/policies/nominatim/)

因为Nominatim是一基于OpenStreetMap data的，因此当访问的数量不大时，它不需要API key，这点非常棒！

### Geocoding in Geopandas

geopands整合了geopy的函数。geopandas的geocode()函数可以geocode一系列的地址，并返回一个包含geometry列的GeoDataframe，列中都是地址对应的点。

数据在这里下载：https://github.com/Automating-GIS-processes/site/tree/master/source/notebooks/L3/data

In [15]:
# Import necessary modules
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point

# Filepath
fp = r"data/addresses.txt"

# Read the data
data = pd.read_csv(fp, sep=';')
data.head()

,id,addr
0,1000,"Itämerenkatu 14, 00101 Helsinki, Finland"
1,1001,"Kampinkuja 1, 00100 Helsinki, Finland"
2,1002,"Kaivokatu 8, 00101 Helsinki, Finland"
3,1003,"Hermannin rantatie 1, 00580 Helsinki, Finland"
4,1005,"Tyynenmerenkatu 9, 00220 Helsinki, Finland"


In [16]:
# Import the geocoding tool
from geopandas.tools import geocode

# Geocode addresses using Nominatim. Remember to provide a custom "application name" in the user_agent parameter!
geo = geocode(data['addr'], provider='nominatim', user_agent='autogis_xx', timeout=4)

In [17]:
geo.head()

,geometry,address
0,POINT (24.91556 60.16320),"Ruoholahti, 14, Itämerenkatu, Ruoholahti, Läns..."
1,POINT (24.93169 60.16902),"Kamppi, 1, Kampinkuja, Kamppi, Eteläinen suurp..."
2,POINT (24.94168 60.16996),"Bangkok9, 8, Kaivokatu, Keskusta, Kluuvi, Etel..."
3,POINT (24.97193 60.19700),"Hermannin rantatie, Kyläsaari, Hermanni, Helsi..."
4,POINT (24.92160 60.15665),"Hesburger, 9, Tyynenmerenkatu, Jätkäsaari, Län..."


### Table join

Table join也是GIS分析中很常用的，也就是基于相同的key属性，把不同表的数据join起来，这在geopandas中可以用merge函数实现。

In [18]:
join = geo.join(data)
join.head()

,geometry,address,id,addr
0,POINT (24.91556 60.16320),"Ruoholahti, 14, Itämerenkatu, Ruoholahti, Läns...",1000,"Itämerenkatu 14, 00101 Helsinki, Finland"
1,POINT (24.93169 60.16902),"Kamppi, 1, Kampinkuja, Kamppi, Eteläinen suurp...",1001,"Kampinkuja 1, 00100 Helsinki, Finland"
2,POINT (24.94168 60.16996),"Bangkok9, 8, Kaivokatu, Keskusta, Kluuvi, Etel...",1002,"Kaivokatu 8, 00101 Helsinki, Finland"
3,POINT (24.97193 60.19700),"Hermannin rantatie, Kyläsaari, Hermanni, Helsi...",1003,"Hermannin rantatie 1, 00580 Helsinki, Finland"
4,POINT (24.92160 60.15665),"Hesburger, 9, Tyynenmerenkatu, Jätkäsaari, Län...",1005,"Tyynenmerenkatu 9, 00220 Helsinki, Finland"


In [19]:
type(join)

geopandas.geodataframe.GeoDataFrame

In [20]:
# Output file path
outfp = r"data/addresses.shp"

# Save to Shapefile
join.to_file(outfp)

## Point in Polygon & Intersect

判断一个点在一个区域内还是区域外，或者看看一条线有没有和另一条线或多边形相交，都是很常见的地理空间运算。这些都是基于位置来选择数据的操作。这些空间查询是很多空间分析典型的第一步。

### 如何判断一个点在不在polygon内

这个问题有一个经典算法[Ray Casting algorithm](https://en.wikipedia.org/wiki/Point_in_polygon#Ray_casting_algorithm)。又很幸运的，the Point in Polygon (PIP) query问题不需要手写该算法，使用 [Shapely’s binary predicates](https://shapely.readthedocs.io/en/stable/manual.html#binary-predicates)可以很快的分析地理对象的空间关系。

Shapely中有两种基本方式处理PIP问题：

1. 使用.within()函数检查一个点是否在polygon内；
2. 使用.contains()检查一个polygon是否包含一个point。

同样这两个函数还可以检查LineString和Polygon是否在一个Polygon内。

In [21]:
from shapely.geometry import Point, Polygon

# Create Point objects
p1 = Point(24.952242, 60.1696017)
p2 = Point(24.976567, 60.1612500)

# Create a Polygon
coords = [(24.950899, 60.169158), (24.953492, 60.169158), (24.953510, 60.170104), (24.950958, 60.169990)]
poly = Polygon(coords)

# Let's check what we have
print(p1)
print(p2)
print(poly)

POINT (24.952242 60.1696017)
POINT (24.976567 60.16125)
POLYGON ((24.950899 60.169158, 24.953492 60.169158, 24.95351 60.170104, 24.950958 60.16999, 24.950899 60.169158))


In [22]:
# Check if p1 is within the polygon using the within function
p1.within(poly)

True

In [23]:
# Check if p2 is within the polygon
p2.within(poly)

False

In [24]:
# Our point
print(p1)

# The centroid
print(poly.centroid)

POINT (24.952242 60.1696017)
POINT (24.95224242849236 60.16960179038188)


In [25]:
# Does polygon contain p1?
poly.contains(p1)

True

In [26]:
# Does polygon contain p2?
poly.contains(p2)

False

如果有许多点，只有一个polygon，然后找出哪些点在polygon，那就循环点，使用within来判断；如果有很多polygons，有一个点，那就用contains函数。